# Database (db) creator and ID trasnsfer to input file


This notebook loads excel files and combines them into a database

In [1]:
# Import packages
import os
from glob import glob
import pandas as pd
import fafbseg

In [2]:
# Some custom functions

def update_dataframe_single_column(source_df, target_df, reference_column):
    # Create a dictionary mapping from the reference column to the source DataFrame
    reference_dict = source_df.groupby(reference_column).first().reset_index().to_dict(orient='records')
    reference_dict = {row[reference_column]: row for row in reference_dict}

    # Update the target DataFrame based on the reference column
    for i, row in target_df.iterrows():
        ref = row[reference_column]
        if ref in reference_dict:
            source_row = reference_dict[ref]
            target_df.loc[i] = source_row

    return target_df

### 1. Loading all data sets of interest in a loop

The original excell files need tzo be stored in the same folder. All files in that folder will be loaded


In [3]:
# Data paths
# Choose path and file
PC_disc = 'D' # why is this necessary? MI
dataPath = f'{PC_disc}:\connectome-analysis-main_thesis\connectome-analysis-main_thesis\database' #\Connectomics-Data\FlyWire\Excels\drive-data-sets\database'  #D:\connectome-analysis-main_thesis\connectome-analysis-main_thesis\database
# remember that f'{} are used for string formatting in python. {PC_disc} is a place holder. I can use that to enter varibales in a string! eg f"My name is {name} and I am {age} years old."
# To use a raw string 'r'\blahblah'
fileName_ls = glob(dataPath +"\\"+ "*.xlsx") # adds all file names from the specific direcotry with the xslx extention. \\ path seperator for windows systems

#Creating the database in a loop
df_ls = [] # empty list
for fileName in fileName_ls:
    print(f'Importing: {fileName}')
    filePath = os.path.join(dataPath,fileName)
    df = pd.read_excel(filePath)
    #Dropping the fisrt row ('asdf' was added as a walk-around to set that column values as type str) ??MI
    if df["seg_id"][0] == 'asdf': 
        df = df.iloc[1: , :] # removes the first row. Why?
        df.reset_index(inplace=True,drop=True)
    df_ls.append(df) # a list of dataframes

Importing: D:\connectome-analysis-main_thesis\connectome-analysis-main_thesis\database\C2 proofreadings.xlsx
Importing: D:\connectome-analysis-main_thesis\connectome-analysis-main_thesis\database\C3 proofreadings.xlsx
Importing: D:\connectome-analysis-main_thesis\connectome-analysis-main_thesis\database\Dm1 proofreadings.xlsx
Importing: D:\connectome-analysis-main_thesis\connectome-analysis-main_thesis\database\Dm10 proofreadings.xlsx
Importing: D:\connectome-analysis-main_thesis\connectome-analysis-main_thesis\database\Dm12 proofreadings.xlsx
Importing: D:\connectome-analysis-main_thesis\connectome-analysis-main_thesis\database\Dm15 proofreadings.xlsx
Importing: D:\connectome-analysis-main_thesis\connectome-analysis-main_thesis\database\Dm2 proofreadings.xlsx
Importing: D:\connectome-analysis-main_thesis\connectome-analysis-main_thesis\database\Dm4 proofreadings.xlsx
Importing: D:\connectome-analysis-main_thesis\connectome-analysis-main_thesis\database\Dm6 proofreadings.xlsx
Importing

### 2. Creating, updating and filtering the database (db) of interest

In [4]:
### Creation
db = pd.concat(df_ls) # new df with concatenated all dfs
print(f'\n\nCell types in the database: {db.symbol.unique()}, total = {len(db.symbol.unique())}') # get the unique neurons, \n\n just two spaced lines when printing
# how does he put the percent scale bar in the printing area?
## Removing any Nan columns
db = db[db["Updated_seg_id"].notna()] # why would there be any nans there?

### Filtering and updating database
## Chossing optic lobe of interest:
_hemisphere = 'R' # variable names can start with an underscore 
db_R = db[db.hemisphere != 'L'].copy() # not equal to L, copies dataframe without the L

## Updating segmnet ids
#Getting the lists of IDs to update
curr_ID_ls = db_R["Updated_seg_id"].tolist() # df to list
curr_ID_ls = db_R["seg_id"].tolist() # isn't he rewriting here?
#Updating all IDs at once
updated_ID_df = fafbseg.flywire.update_ids(curr_ID_ls, stop_layer=2, supervoxels=None, timestamp=None, 
                                           dataset='production', progress=True)
db_R['Updated_seg_id'] = updated_ID_df['new_id'].astype(str).tolist() # astype(str) converts to string 
db_R['Updated_confidence'] = updated_ID_df['confidence'].astype(str).tolist() # what is confidence?



Cell types in the database: ['C2' 'C3' 'Dm1' 'Dm10' 'Dm12' 'Dm15' 'Dm2a' 'Dm2b' 'Dm2c' 'Dm4' 'Dm6'
 'L1' 'L2' 'L3' 'L5' 'Me-Lo-2-N.I.' 'Mi1' 'Mi10' 'Mi13' 'Mi4' 'Mi9' 'ML1'
 'Pm1_Tm1' 'Pm1_Tm2' 'Pm1a_Tm1' 'Pm1a_Mi1' 'Pm2' 'Pm3' 'Pm4' 'OA-AL2b2-L1'
 'OA-AL2b2-L2' 'OA-AL2b2-R1' 'OA-AL2b2-R2' 'CT1' 'pMP7' 'PS125' 'R7' 'R8'
 'T1' 'T3' 'T4a' 'T4b' 'T4c' 'T4d' 'Tm1' 'Tm1/2' 'Tm16' 'Tm2' 'Tm20'
 'Tm20?' 'Tm3' 'Tm4' 'Tm4a' 'Tm9' 'Tm10' 'Tm11' 'Tm12' 'TmY10' 'TmY15'
 'TmY17'], total = 60


Updating:   0%|          | 0/21022 [00:00<?, ?it/s]

In [5]:
db_R

,XYZ-ME,symbol,XYZ-LO,quality comment,hemisphere,seg_id,identified_in,Updated_seg_id,Update_confidence,updated_seg_id,...,lab authorship (Y/N),patch_id,column_id,dark_neurons (Y/N),bad_quality_rim (Y/N),healthy_L3 (Y/N),detached_lamina (Y/N),Updated_confidence,Mi4-connected,dorso-ventral
250,NaN,C2,NaN,strange dendrite,R,720575940649255801,"Fischbach & Dittrich, 1989",720575940647498019,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN
251,NaN,C2,NaN,strange dendrite,R,720575940617134678,"Fischbach & Dittrich, 1989",720575940625965756,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.99,NaN,NaN
252,NaN,C2,NaN,strange dendrite,R,720575940616540043,"Fischbach & Dittrich, 1989",720575940616540043,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN
253,NaN,C2,NaN,no axon terminals,R,720575940612496497,"Fischbach & Dittrich, 1989",720575940612496497,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN
254,NaN,C2,NaN,strange dendrite,R,720575940618589953,"Fischbach & Dittrich, 1989",720575940618589953,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
151,NaN,TmY17,NaN,NaN,R,720575940633505934,NaN,720575940633505934,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN
152,NaN,TmY17,NaN,NaN,R,720575940651270902,NaN,720575940651270902,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN
153,NaN,TmY17,NaN,NaN,R,720575940625696637,NaN,720575940625696637,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN
154,NaN,TmY17,NaN,NaN,R,720575940625313428,NaN,720575940625313428,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN


In [20]:
db_R['symbol'].unique()
db_R

,XYZ-ME,symbol,XYZ-LO,quality comment,hemisphere,seg_id,identified_in,Updated_seg_id,Update_confidence,updated_seg_id,...,patch_id,column_id,dark_neurons (Y/N),bad_quality_rim (Y/N),healthy_L3 (Y/N),detached_lamina (Y/N),Updated_confidence,Mi4-connected,dorso-ventral,Updated_seg_ids
250,NaN,C2,NaN,strange dendrite,R,720575940649255801,"Fischbach & Dittrich, 1989",720575940647498019,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,720575940647498019
251,NaN,C2,NaN,strange dendrite,R,720575940617134678,"Fischbach & Dittrich, 1989",720575940625965756,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.99,NaN,NaN,720575940625965756
252,NaN,C2,NaN,strange dendrite,R,720575940616540043,"Fischbach & Dittrich, 1989",720575940616540043,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,720575940616540043
253,NaN,C2,NaN,no axon terminals,R,720575940612496497,"Fischbach & Dittrich, 1989",720575940612496497,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,720575940612496497
254,NaN,C2,NaN,strange dendrite,R,720575940618589953,"Fischbach & Dittrich, 1989",720575940618589953,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,720575940618589953
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
151,NaN,TmY17,NaN,NaN,R,720575940633505934,NaN,720575940633505934,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,720575940633505934
152,NaN,TmY17,NaN,NaN,R,720575940651270902,NaN,720575940651270902,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,720575940651270902
153,NaN,TmY17,NaN,NaN,R,720575940625696637,NaN,720575940625696637,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,720575940625696637
154,NaN,TmY17,NaN,NaN,R,720575940625313428,NaN,720575940625313428,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,720575940625313428


### 3. Creating a input file of interest

In [5]:
#Selecting postsynaptic neuron of interest
neuron = 'Mi1'
neuron_df = db_R[db_R['symbol'] == neuron].copy()

#Filtering for valid segment ids based on a given criteria
neuron_selected_df = neuron_df.copy()
neuron_selected_df = neuron_selected_df.copy() # but I will just take all MI
# neuron_selected_df[neuron_selected_df['backbone proofread (Y/N)'] == 'Y'].copy()
neuron_selected_df = neuron_selected_df.copy() # Temp just for C3 neuron_selected_df.iloc[:135].copy()
#neuron_selected_df = neuron_selected_df[neuron_selected_df['inputs_proofread (Y/N)'] == 'N'].copy()

print(f'\n\nTotal number of postsynaptic cells: {len(neuron_selected_df)}\n\n')

ID_ls = neuron_selected_df['Updated_seg_id'].tolist()
####temp_maria
#Mi1s_patch1 = [720575940621959185,720575940610177941,720575940621647996,720575940636894022,720575940610254037,720575940639404605,720575940629418110,720575940603042272,720575940650403449,720575940617839832,720575940615088034,720575940632717359,720575940630171207,720575940644391112,720575940625269822,720575940609463620]
#Mi1s_patch2 = [720575940611524835,720575940631147484,720575940610460402,720575940630964551,720575940610772174,720575940610674090,720575940612556438,720575940637260133,720575940629763663,720575940610953478,720575940616367133,720575940629678124,720575940632107033,720575940628892432,720575940610952198,720575940614282134]
#Mi1s_patch3 =[720575940634823822,720575940615420358,720575940631379512,720575940624234644,720575940632700256,720575940633027539,720575940624177900,720575940622954189,720575940625383944,720575940627733447,720575940621505718,720575940626810442,720575940623328184,720575940627160143,720575940628618920,720575940637643950]
#Mi1s_patch4 =[720575940640572733,720575940622304895,720575940630827339,720575940639250741,720575940613120741,720575940636425870,720575940619977087,720575940631894753,720575940622933444,720575940631103007,720575940626104648,720575940609555406,720575940628157595,720575940627970122,720575940626900086,720575940627669239]
#Mi1s_patch5
#Mi1s_patch6
#Mi1s =[720575940640572733,720575940622304895,720575940630827339,720575940639250741,720575940613120741,720575940636425870,720575940619977087,720575940631894753,720575940622933444,720575940631103007,720575940626104648,720575940609555406,720575940628157595,720575940627970122,720575940626900086,720575940627669239]
####temp_maria_end
#Fetching the neuron's inputs and putputs
neurons_inputs = fafbseg.flywire.synapses.fetch_synapses(ID_ls, pre=False, post=True, attach=True,  # Mi1s
                                             min_score=50, clean=True, transmitters=False, 
                                             neuropils=True, batch_size=30, 
                                             dataset='production', progress=True,mat= "latest")

neurons_outputs = fafbseg.flywire.synapses.fetch_synapses(ID_ls, pre=True, post=False, attach=True, 
                                             min_score=50, clean=True, transmitters=False, 
                                             neuropils=True, batch_size=30, 
                                             dataset='production', progress=True,mat= "latest") # here seb had mat = "live" but produces an error to me possibly from the cave version


#Counting inputs per ID, option joining dataframes
final_input_df = pd.DataFrame()
for n in neurons_inputs['post'].unique():
    inputs_count = {} # creates an empty dictionary
    curr_inputs = neurons_inputs[neurons_inputs['post'] == n]
    inputs_str = curr_inputs.applymap(str)
    
    for c in inputs_str['pre'].to_list():
        inputs_count[c] = inputs_count.get(c, 0) + 1 # access the dictionary with c as key
    input_count_df = pd.DataFrame(inputs_count, index=[0])
    input_count_df = input_count_df.T
    input_count_df.rename(columns={0: "counts"},inplace=True)
    input_count_df.index.names = ['presynaptic_ID']
    input_count_df = input_count_df.sort_values(by="counts",ascending=False)
    input_count_df['postsynaptic_ID'] = inputs_str['post'].to_list()[0:len(input_count_df)]
    final_input_df = final_input_df.append(input_count_df)
    #print(f'Counting done for: {n}')
input_count_str_df = final_input_df.applymap(str)
print('INPUTS: ')
display(input_count_str_df.head())


#Counting outputs per ID, option joining dataframes
final_output_df = pd.DataFrame()
for n in neurons_outputs['pre'].unique():
    outputs_count = {}
    curr_outputs = neurons_outputs[neurons_outputs['pre'] == n]
    outputs_str = curr_outputs.applymap(str)
    
    for c in outputs_str['post'].to_list():
        outputs_count[c] = outputs_count.get(c, 0) + 1
    output_count_df = pd.DataFrame(outputs_count, index=[0])
    output_count_df = output_count_df.T
    output_count_df.rename(columns={0: "counts"},inplace=True)
    output_count_df.index.names = ['postsynaptic_ID']
    output_count_df = output_count_df.sort_values(by="counts",ascending=False)
    output_count_df['presynaptic_ID'] = outputs_str['pre'].to_list()[0:len(output_count_df)]
    final_output_df = final_output_df.append(output_count_df)
    #print(f'Counting done for: {n}')
output_count_str_df = final_output_df.applymap(str)
print('OUTPUTS: ')
display(output_count_str_df.head())




Total number of postsynaptic cells: 797




Fetching synapses:   0%|          | 0/27 [00:00<?, ?it/s]

Using deprecated pyarrow serialization method, please upgrade CAVEClient>=5.9.0 with pip install --upgrade caveclient
Using deprecated pyarrow serialization method, please upgrade CAVEClient>=5.9.0 with pip install --upgrade caveclient
Using deprecated pyarrow serialization method, please upgrade CAVEClient>=5.9.0 with pip install --upgrade caveclient
Using deprecated pyarrow serialization method, please upgrade CAVEClient>=5.9.0 with pip install --upgrade caveclient
Using deprecated pyarrow serialization method, please upgrade CAVEClient>=5.9.0 with pip install --upgrade caveclient
Using deprecated pyarrow serialization method, please upgrade CAVEClient>=5.9.0 with pip install --upgrade caveclient
Using deprecated pyarrow serialization method, please upgrade CAVEClient>=5.9.0 with pip install --upgrade caveclient
Using deprecated pyarrow serialization method, please upgrade CAVEClient>=5.9.0 with pip install --upgrade caveclient
Using deprecated pyarrow serialization method, please up

Fetching synapses:   0%|          | 0/27 [00:00<?, ?it/s]

Using deprecated pyarrow serialization method, please upgrade CAVEClient>=5.9.0 with pip install --upgrade caveclient
Using deprecated pyarrow serialization method, please upgrade CAVEClient>=5.9.0 with pip install --upgrade caveclient
Using deprecated pyarrow serialization method, please upgrade CAVEClient>=5.9.0 with pip install --upgrade caveclient
Using deprecated pyarrow serialization method, please upgrade CAVEClient>=5.9.0 with pip install --upgrade caveclient
Using deprecated pyarrow serialization method, please upgrade CAVEClient>=5.9.0 with pip install --upgrade caveclient
Using deprecated pyarrow serialization method, please upgrade CAVEClient>=5.9.0 with pip install --upgrade caveclient
Using deprecated pyarrow serialization method, please upgrade CAVEClient>=5.9.0 with pip install --upgrade caveclient
Using deprecated pyarrow serialization method, please upgrade CAVEClient>=5.9.0 with pip install --upgrade caveclient
Using deprecated pyarrow serialization method, please up

INPUTS: 


,counts,postsynaptic_ID
presynaptic_ID,,
720575940637490266,150,720575940632107033
720575940628030724,64,720575940632107033
720575940625460203,19,720575940632107033
720575940618253529,18,720575940632107033
720575940621502068,17,720575940632107033


OUTPUTS: 


,counts,presynaptic_ID
postsynaptic_ID,,
720575940628053009,65,720575940625276958
720575940621802049,62,720575940625276958
720575940627907358,57,720575940625276958
720575940627210084,38,720575940625276958
720575940610876258,34,720575940625276958


### 4. Adding useful information to the data frame

In [6]:
## For INPUTS


# Selecting dataframe
#Updating the IDs via Fafbseg
partner_ID = input_count_str_df.index.tolist()
updated_ID_df = fafbseg.flywire.update_ids(partner_ID, stop_layer=2, supervoxels=None, timestamp=None, dataset='production', progress=True)
partner_ID_ls = updated_ID_df["new_id"].tolist()

# Identifying user-based annotations about cell identity

identification_df = fafbseg.flywire.find_celltypes(partner_ID_ls, user=None, exact=False, case=False, regex=True, update_roots=False)
identification_no_duplicates_df = identification_df.drop_duplicates(subset='pt_root_id', keep='last', inplace=False, ignore_index=False).copy()

# Adding info to the current data set

# Selecting dataframes and resetting index
source_df = identification_no_duplicates_df.copy()
source_df.reset_index(inplace = True, drop = True)
target_df = input_count_str_df.copy()
target_df.reset_index(inplace = True, drop = True)


# Adding columns for the function to properly work
target_df['presynaptic_ID'] = input_count_str_df.index.astype(str)
source_df['presynaptic_ID'] = identification_no_duplicates_df['pt_root_id'].tolist()
target_df['guess'] = None
source_df['guess'] = identification_no_duplicates_df['tag'].tolist()
target_df['author'] = None
source_df['author'] = identification_no_duplicates_df['user_id'].tolist()

# Function inputs
source_cols = ['guess', 'author','presynaptic_ID']
target_cols = ['guess', 'author', 'presynaptic_ID']
reference_column = 'presynaptic_ID'

source_df = source_df[source_cols].copy()
target_df = target_df[source_cols].copy()

source_df = source_df.astype(str)
target_df = target_df.astype(str)


# Running the function and completing the dataset
result_input_df = update_dataframe_single_column(source_df, target_df,reference_column)
result_input_df['counts'] = input_count_str_df['counts'].tolist()
result_input_df['postsynaptic_ID'] = input_count_str_df['postsynaptic_ID'].tolist()
result_input_df.head()

Updating:   0%|          | 0/64777 [00:00<?, ?it/s]

Using deprecated pyarrow serialization method, please upgrade CAVEClient>=5.9.0 with pip install --upgrade caveclient


,guess,author,presynaptic_ID,counts,postsynaptic_ID
0,L1,94,720575940637490266,150,720575940632107033
1,L5,94,720575940628030724,64,720575940632107033
2,Dm1; Distal medullary amacrine neuron 1; FBbt_...,392,720575940625460203,19,720575940632107033
3,Proximal medulla 2; Pm2,17,720575940618253529,18,720575940632107033
4,L3,94,720575940621502068,17,720575940632107033


In [7]:
## For OUTPUTS

# Selecting dataframe
#Updating the IDs via Fafbseg
partner_ID = output_count_str_df.index.tolist()
updated_ID_df = fafbseg.flywire.update_ids(partner_ID, stop_layer=2, supervoxels=None, timestamp=None, dataset='production', progress=True)
partner_ID_ls = updated_ID_df["new_id"].tolist()

# Identifying user-based annotations about cell identity

identification_df = fafbseg.flywire.find_celltypes(partner_ID_ls, user=None, exact=False, case=False, regex=True, update_roots=False)
identification_no_duplicates_df = identification_df.drop_duplicates(subset='pt_root_id', keep='last', inplace=False, ignore_index=False).copy()

# Adding info to the current data set

# Selecting dataframes and resetting index
source_df = identification_no_duplicates_df.copy()
source_df.reset_index(inplace = True, drop = True)
target_df = output_count_str_df.copy()
target_df.reset_index(inplace = True, drop = True)


# Adding columns for the function to properly work
target_df['postsynaptic_ID'] = output_count_str_df.index.astype(str)
source_df['postsynaptic_ID'] = identification_no_duplicates_df['pt_root_id'].tolist()
target_df['guess'] = None
source_df['guess'] = identification_no_duplicates_df['tag'].tolist()
target_df['author'] = None
source_df['author'] = identification_no_duplicates_df['user_id'].tolist()

# Function inputs
source_cols = ['guess', 'author','postsynaptic_ID']
target_cols = ['guess', 'author', 'postsynaptic_ID']
reference_column = 'postsynaptic_ID'

source_df = source_df[source_cols].copy()
target_df = target_df[source_cols].copy()

source_df = source_df.astype(str)
target_df = target_df.astype(str)


# Running the function and compleating the dataset
result_output_df = update_dataframe_single_column(source_df, target_df,reference_column)
result_output_df['counts'] = output_count_str_df['counts'].tolist()
result_output_df['presynaptic_ID'] = output_count_str_df['presynaptic_ID'].tolist()
result_output_df.head()

Checking:   0%|          | 0/425988 [00:00<?, ?it/s]

Updating:   0%|          | 0/425988 [00:00<?, ?it/s]

Checking:   0%|          | 0/425988 [00:00<?, ?it/s]

,guess,author,postsynaptic_ID,counts,presynaptic_ID
0,Proximal medulla 1a; Pm1a,17,720575940628053009,65,720575940625276958
1,Proximal medulla 1a; Pm1a,17,720575940621802049,62,720575940625276958
2,Pm4; Proximal medullary amacrine neuron 4; FBb...,392,720575940627907358,57,720575940625276958
3,Proximal medulla 2; Pm2,17,720575940627210084,38,720575940625276958
4,T4d; FBbt_00003735,392,720575940610876258,34,720575940625276958


### 5. Transfering information from the main database (db)

In [8]:
## For INPUTS

# Matcing data types
db_R['Updated_seg_id'] = db_R['Updated_seg_id'].astype(str)
result_input_df['presynaptic_ID'] = result_input_df['presynaptic_ID'].astype(str)

# Merging the DataFrames based on common values
merged_input_df = pd.merge(result_input_df, db_R[['Updated_seg_id', 'symbol']], left_on='presynaptic_ID', right_on='Updated_seg_id', how='left')

# Drop the extra 'seg_id' column
merged_input_df.drop(columns=['Updated_seg_id'], inplace=True)

# Display the merged DataFrame
print('For INPUTS')
display(merged_input_df)


## For OUTPUTS

# Matcing data types
db_R['Updated_seg_id'] = db_R['Updated_seg_id'].astype(str)
result_output_df['postynaptic_ID'] = result_output_df['postsynaptic_ID'].astype(str)

# Merging the DataFrames based on common values
merged_output_df = pd.merge(result_output_df, db_R[['Updated_seg_id', 'symbol']], left_on='postsynaptic_ID', right_on='Updated_seg_id', how='left')

# Drop the extra 'seg_id' column
merged_output_df.drop(columns=['Updated_seg_id'], inplace=True)

# Display the merged DataFrame
print('For OUTPUTS')
display(merged_output_df)

For INPUTS


,guess,author,presynaptic_ID,counts,postsynaptic_ID,symbol
0,L1,94,720575940637490266,150,720575940632107033,L1
1,L5,94,720575940628030724,64,720575940632107033,L5
2,Dm1; Distal medullary amacrine neuron 1; FBbt_...,392,720575940625460203,19,720575940632107033,Dm1
3,Proximal medulla 2; Pm2,17,720575940618253529,18,720575940632107033,Pm2
4,L3,94,720575940621502068,17,720575940632107033,L3
...,...,...,...,...,...,...
67083,None,None,720575940604535426,1,720575940625577912,NaN
67084,Dm1; Distal medullary amacrine neuron 1; FBbt_...,392,720575940621460928,1,720575940625577912,Dm1
67085,Dm12; Distal medullary amacrine neuron 12; FBb...,392,720575940625026743,1,720575940625577912,Dm12
67086,Dm17,100,720575940632069583,1,720575940625577912,NaN


For OUTPUTS


,guess,author,postsynaptic_ID,counts,presynaptic_ID,postynaptic_ID,symbol
0,Proximal medulla 1a; Pm1a,17,720575940628053009,65,720575940625276958,720575940628053009,Pm1a_Tm1
1,Proximal medulla 1a; Pm1a,17,720575940628053009,65,720575940625276958,720575940628053009,Pm1a_Mi1
2,Proximal medulla 1a; Pm1a,17,720575940621802049,62,720575940625276958,720575940621802049,Pm1a_Mi1
3,Pm4; Proximal medullary amacrine neuron 4; FBb...,392,720575940627907358,57,720575940625276958,720575940627907358,Pm4
4,Proximal medulla 2; Pm2,17,720575940627210084,38,720575940625276958,720575940627210084,Pm2
...,...,...,...,...,...,...,...
428072,None,None,720575940398684577,1,720575940625577912,720575940398684577,NaN
428073,None,None,720575940504243913,1,720575940625577912,720575940504243913,NaN
428074,None,None,720575940604567426,1,720575940625577912,720575940604567426,NaN
428075,None,None,720575940398672033,1,720575940625577912,720575940398672033,NaN


### 6. Adding more useful information for later analysis

In [9]:
## For INPUTS

# Matcing data types
neuron_df['Updated_seg_id'] = neuron_df['Updated_seg_id'].astype(str)
merged_input_df['postsynaptic_ID'] = merged_input_df['postsynaptic_ID'].astype(str)

# Merging the DataFrames based on common values
merged_input_2_df = pd.merge(merged_input_df, neuron_df[['Updated_seg_id', 'optic_lobe_id','dorso-ventral']], left_on='postsynaptic_ID', right_on='Updated_seg_id', how='left')

# Drop the extra 'seg_id' column
merged_input_2_df.drop(columns=['Updated_seg_id'], inplace=True)

# Display the merged DataFrame
print('For INPUTS:')
display(merged_input_2_df)


## For OUTPUTS

# Matcing data types
neuron_df['Updated_seg_id'] = neuron_df['Updated_seg_id'].astype(str)
merged_output_df['presynaptic_ID'] = merged_output_df['presynaptic_ID'].astype(str)

# Merging the DataFrames based on common values
merged_output_2_df = pd.merge(merged_output_df, neuron_df[['Updated_seg_id', 'optic_lobe_id','dorso-ventral']], left_on='presynaptic_ID', right_on='Updated_seg_id', how='left')

# Drop the extra 'seg_id' column
merged_output_2_df.drop(columns=['Updated_seg_id'], inplace=True)

# Display the merged DataFrame
print('For OUTPUTS:')
display(merged_output_2_df)

For INPUTS:


,guess,author,presynaptic_ID,counts,postsynaptic_ID,symbol,optic_lobe_id,dorso-ventral
0,L1,94,720575940637490266,150,720575940632107033,L1,Mi1-R15,NaN
1,L5,94,720575940628030724,64,720575940632107033,L5,Mi1-R15,NaN
2,Dm1; Distal medullary amacrine neuron 1; FBbt_...,392,720575940625460203,19,720575940632107033,Dm1,Mi1-R15,NaN
3,Proximal medulla 2; Pm2,17,720575940618253529,18,720575940632107033,Pm2,Mi1-R15,NaN
4,L3,94,720575940621502068,17,720575940632107033,L3,Mi1-R15,NaN
...,...,...,...,...,...,...,...,...
67083,None,None,720575940604535426,1,720575940625577912,NaN,Mi1-R782,NaN
67084,Dm1; Distal medullary amacrine neuron 1; FBbt_...,392,720575940621460928,1,720575940625577912,Dm1,Mi1-R782,NaN
67085,Dm12; Distal medullary amacrine neuron 12; FBb...,392,720575940625026743,1,720575940625577912,Dm12,Mi1-R782,NaN
67086,Dm17,100,720575940632069583,1,720575940625577912,NaN,Mi1-R782,NaN


For OUTPUTS:


,guess,author,postsynaptic_ID,counts,presynaptic_ID,postynaptic_ID,symbol,optic_lobe_id,dorso-ventral
0,Proximal medulla 1a; Pm1a,17,720575940628053009,65,720575940625276958,720575940628053009,Pm1a_Tm1,Mi1-R17,NaN
1,Proximal medulla 1a; Pm1a,17,720575940628053009,65,720575940625276958,720575940628053009,Pm1a_Mi1,Mi1-R17,NaN
2,Proximal medulla 1a; Pm1a,17,720575940621802049,62,720575940625276958,720575940621802049,Pm1a_Mi1,Mi1-R17,NaN
3,Pm4; Proximal medullary amacrine neuron 4; FBb...,392,720575940627907358,57,720575940625276958,720575940627907358,Pm4,Mi1-R17,NaN
4,Proximal medulla 2; Pm2,17,720575940627210084,38,720575940625276958,720575940627210084,Pm2,Mi1-R17,NaN
...,...,...,...,...,...,...,...,...,...
428072,None,None,720575940398684577,1,720575940625577912,720575940398684577,NaN,Mi1-R782,NaN
428073,None,None,720575940504243913,1,720575940625577912,720575940504243913,NaN,Mi1-R782,NaN
428074,None,None,720575940604567426,1,720575940625577912,720575940604567426,NaN,Mi1-R782,NaN
428075,None,None,720575940398672033,1,720575940625577912,720575940398672033,NaN,Mi1-R782,NaN


### Saving 

In [10]:
# Saving data in your computer
PC_disc = 'D'
outDir = f'{PC_disc}:\connectome-analysis-main_thesis\connectome-analysis-main_thesis\Mi1_outputs' # YOUR-PATH for saving excel file
save_excel_file = True

import datetime
x = datetime.datetime.now()
date_str = x.strftime("%d") + x.strftime("%b") + x.strftime("%Y")

if save_excel_file: 
    ## Input count
    file_name = f'{neuron}_neurons_input_count_{_hemisphere}_{date_str}.xlsx'
    savePath = os.path.join(outDir, file_name)
    merged_input_2_df.to_excel(savePath, sheet_name='Buhmann synapses, inputs')
    
    ## Output count
    file_name = f'{neuron}_neurons_output_count_{_hemisphere}_{date_str}.xlsx'
    savePath = os.path.join(outDir, file_name)
    merged_output_2_df.to_excel(savePath, sheet_name='Buhmann synapses, outputs')